# Installing Dependencies

In [ ]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
%pip install scipy

# Ex 1.

Compare the performance of a $kNN$ with $k = 5$ and a naïve Bayes with Gaussian \
assumption (consider all remaining parameters as default):

### a)
Plot two boxplots with the fold accuracies for each classifier. Is there one \
more stable than the other regarding performance? Why do you think that is the \
case? Explain. 
> (**Refer to the report for the explanation**)

In [ ]:
# Code for a)

### b)
Report the accuracy of both models, this time scaling the data with a \
Min-Max scaler before training the models. Explain the impact that this \
preprocessing step has on the performance of each model, providing an \
explanation for the results. 
> (**Refer to the report for the explanation**)

In [ ]:
# Code for b)

### c)
Using `scipy`, test the hypothesis “the $kNN$ model is statistically superior to \
naïve Bayes regarding accuracy”, asserting whether it is true.
> (**Refer to the report for the explanation**)

In [ ]:
# Code for c)

# Ex 2.
Using a 80-20 train-test split, vary the number of neighbors of a $kNN$ classifier using \
$k = \{1, 5, 10, 20, 30\}$. Additionally, for each $k$, train one classifier using uniform weights \
and distance weights.

### a)
Plot the train and test accuracy for each model

In [ ]:
# Code for a)

### b)
Explain the impact of increasing the neighbors on the generalization ability of \
the models.
> (**Refer to the report for the explanation**)

# Ex 3.
Considering the unique properties of the `heart-disease.csv` dataset, identify two
possible difficulties of the naïve Bayes model used in the previous exercises when learning
from the given dataset.
> (**Refer to the report for the explanation**)